<a href="https://colab.research.google.com/github/Abdibaset/ML_CS74/blob/main/Sentiment_Analysis_AmazonReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.utils import validation
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, f1_score, accuracy_score
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
from sklearn.linear_model import Perceptron

# **Metrics**

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score
def evaluationMetrics(y_actual, y_predicted):
  confusion_matrix_result = confusion_matrix(y_actual, y_predicted)
  roc_auc_result = roc_auc_score(y_actual, y_predicted)
  accuracy_result = accuracy_score(y_actual, y_predicted)
  f1_score_result = f1_score(y_actual, y_predicted)

  print("Confusion Matrix: ", confusion_matrix_result, "\nroc_auc: ", roc_auc_result, "\naccuracy: ", accuracy_result, "\nf1_score: ", f1_score_result)

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data = train_data[['summary', 'reviewText', 'overall', 'verified', 'category']]
test_data = test_data[['summary', 'reviewText', 'id', 'verified', 'category']]

## convert to str
train_data['verified'] = train_data['verified'].apply(lambda x: "true" if x else "false")
test_data['verified'] = test_data['verified'].apply(lambda x: "true" if x else "false")

## text features
important_text_features_train = train_data["reviewText"].fillna('') + ' ' +  train_data["summary"].fillna('') + ' ' + train_data["verified"].fillna('') + ' ' + train_data["category"].fillna('')
important_text_features_test = test_data["reviewText"].fillna('') + ' '+ test_data["summary"].fillna('') + test_data["verified"].fillna('') + ' ' + test_data["category"].fillna('')

target_label = train_data['overall'].apply(lambda f: 1 if f > 3 else 0)
train_data.head()


<ipython-input-66-1402d56263b2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['verified'] = test_data['verified'].apply(lambda x: "true" if x else "false")


,summary,reviewText,overall,verified,category
0,"All fake reviews, beware.",all of the reviews for this product are fake.,1,false,automotive
1,One Star,wrong part. our fault.,1,true,automotive
2,One Star,this wire set it really sucks!!!,1,true,automotive
3,One Star,"first use, it leaked instantly. even at 5 buck...",1,true,automotive
4,One Star,didn't fit,1,true,automotive


# **Preprocessing Text**

In [ ]:
import os
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer

nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def clean_text(text):
    text = text.lower()
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    return text

def tokenize(text):
    return word_tokenize(text)

def lemmatize(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

def stem(tokens):
    return [stemmer.stem(token) for token in tokens]

def clean_text_features(text_features):
    cleaned_features = []
    for feature in text_features:
        text = clean_text(feature)
        tokens = tokenize(text)
        tokens = lemmatize(tokens)
        tokens = stem(tokens)
        cleaned_features.append(' '.join(tokens))
    return cleaned_features

important_text_features_train = clean_text_features(important_text_features_train)
important_text_features_test = clean_text_features(important_text_features_test)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **tfidf vectorization on text features**

In [ ]:
# better without removing stop words
vectorizer = TfidfVectorizer(
    ngram_range = (1, 2),
)
train_text_features = vectorizer.fit_transform(important_text_features_train)
test_text_features = vectorizer.transform(important_text_features_test)
train_set,validation_set, train_target, validation_target = train_test_split(train_text_features, target_label, test_size = 0.2, random_state = 27)

# **Logistic Regression Model**

In [ ]:
from sklearn.linear_model import LogisticRegression

logReg = LogisticRegression()

# parameter tuning
parameters = {
    "max_iter": [850, 1000],
    "C": [3.5, 4.0,  5.0],
      "solver": ["lbfgs", "saga"]
}

## search for best params
grid_search = GridSearchCV(logReg, parameters, cv = 5, scoring="f1_macro")
grid_search.fit(train_set, train_target)
best_params = grid_search.best_params_
print('best grid parameters', best_params)

## retrain model using the best params and evaluating its performance
best_logReg = LogisticRegression(C=best_params["C"], max_iter=best_params["max_iter"], solver=best_params["solver"])
best_logReg.fit(train_set, train_target)
validation_pred = best_logReg.predict(validation_set)
evaluationMetrics(validation_target, validation_pred)

## predict test and save data
test_predicted = best_logReg.predict(test_text_features)
logReg_submission = {'preds': test_predicted, 'id':test_data['id']}
logReg_submission_df = pd.DataFrame(logReg_submission)
print(logReg_submission_df['preds'].value_counts())
logReg_submission_df.to_csv('logReg_submission.csv', index=False)


best grid parameters {'C': 5.0, 'max_iter': 1000, 'solver': 'saga'}
Confusion Matrix:  [[3384  230]
 [ 486 1738]] 
roc_auc:  0.8589167127836193 
accuracy:  0.8773552586502227 
f1_score:  0.8291984732824428
0    2833
1    1667
Name: preds, dtype: int64


# **Naive Bayes Model**

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

# parameter tuning
param_grid = {
    'alpha': [0.1, 0.2, 0.3, 0.4]
}

## search for best params
naive_bayes = MultinomialNB()
grid_search = GridSearchCV(estimator=naive_bayes, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(train_set, train_target)
best_params = grid_search.best_params_
print('best grid parameters', best_params)

## retrain model using the best params and evaluating its performance
best_NB = MultinomialNB(alpha=best_params['alpha'])
best_NB.fit(train_set, train_target)
validation_pred = best_NB.predict(validation_set)
evaluationMetrics(validation_target, validation_pred)

# predict test classes and save the results to a CSV file
test_predicted = best_NB.predict(test_text_features)
first_submission = {'preds': test_predicted, 'id': test_data['id']}
first_submission_df = pd.DataFrame(first_submission)
print(first_submission_df['preds'].value_counts())
first_submission_df.to_csv('nb_submission.csv', index=False)

best grid parameters {'alpha': 0.1}
Confusion Matrix:  [[3469  145]
 [ 837 1387]] 
roc_auc:  0.7917646651909243 
accuracy:  0.8317917094895512 
f1_score:  0.7385516506922257
0    3543
1     957
Name: preds, dtype: int64


In [ ]:
perceptron = Perceptron()
param_grid = {
    'alpha': [0.1, 0.2, 0.3, 0.5, 1],
    'max_iter': [850, 1000, 2000],
}

## search for best params
grid_search = GridSearchCV(perceptron, param_grid, cv=15, scoring='f1_macro', n_jobs=-1)
grid_search.fit(train_set, train_target)
best_params = grid_search.best_params_
print(best_params)

## retrain model using the best params and evaluating its performance
best_perceptron = Perceptron(alpha=best_params["alpha"], max_iter=best_params["max_iter"])
best_perceptron.fit(train_set, train_target)
val_predicted = best_perceptron.predict(validation_set)
evaluationMetrics(validation_target, val_predicted)

# predict test classes and save the results to a CSV file
test_predicted = best_perceptron.predict(test_text_features)
submission_data = {'preds': test_predicted, 'id': test_data['id']}
submission_df = pd.DataFrame(submission_data)
print(submission_df['preds'].value_counts())
submission_df.to_csv('perceptron_submission.csv', index=False)

{'alpha': 0.1, 'max_iter': 850}
Confusion Matrix:  [[3095  519]
 [ 302 1922]] 
roc_auc:  0.8603002213613724 
accuracy:  0.8593696471394313 
f1_score:  0.8240085744908895
0    2528
1    1972
Name: preds, dtype: int64


# **Preprocessing for Sentiment Analysis**

In [ ]:
from posixpath import split
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import StandardScaler

vectorizer = TfidfVectorizer(
    ngram_range = (1, 2)
)

train_text_features_SA = vectorizer.fit_transform(important_text_features_train)
test_text_features_SA = vectorizer.transform(important_text_features_test)

## get scores for the sentiment analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()
train_features_SA = pd.DataFrame([sentiment_analyzer.polarity_scores(feature) for feature in important_text_features_train])
test_features_SA = pd.DataFrame([sentiment_analyzer.polarity_scores(feature) for feature in important_text_features_test])

print('sentence', important_text_features_train[0], '\n', sentiment_analyzer.polarity_scores(important_text_features_train[0]))

# combine tfidf and sentiment analyzer data
X_train_combined = hstack([train_text_features_SA, train_features_SA])
X_test_combined = hstack([test_text_features_SA, test_features_SA])

x_sa_train, x_sa_validation, y_sa_target_train, y_sa_target_validation = train_test_split(X_train_combined, target_label, test_size=0.2, random_state=27)


sentence all of the review for thi product are fake all fake review bewar fals automot 
 {'neg': 0.323, 'neu': 0.677, 'pos': 0.0, 'compound': -0.7351}


# **Sentiment Analysis + Regression Model**

In [ ]:
from imblearn.over_sampling import RandomOverSampler

param_grid = {
    'C': [3.5, 4, 5],
    'max_iter': [2000],
    'solver': ['saga', 'lbfgs']
}
logReg = LogisticRegression()

## search for best params
grid_search = GridSearchCV(logReg, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(x_sa_train, y_sa_target_train)
best_params = grid_search.best_params_
print(best_params)

## retrain model using the best params and evaluating its performance
best_sa_logReg = LogisticRegression(C=best_params["C"], max_iter=best_params["max_iter"], solver=best_params["solver"])
best_sa_logReg.fit(x_sa_train, y_sa_target_train)
y_pred = best_sa_logReg.predict(x_sa_validation)
evaluationMetrics(y_sa_target_validation, y_pred)

# predict test classes and save the results to a CSV file
test_predicted = best_sa_logReg.predict(X_test_combined)
submission_data = {'preds': test_predicted, 'id': test_data['id']}
submission_df = pd.DataFrame(submission_data)
print(submission_df['preds'].value_counts())
submission_df.to_csv('log_sa_submission.csv', index=False)

{'C': 5, 'max_iter': 2000, 'solver': 'saga'}


# **Sentiment Analysis + Perceptron**

In [ ]:
perceptron = Perceptron()
param_grid = {
    'alpha': [0.1, 0.2, 0.3, 0.4],
    'max_iter': [2000, 3000],
}


grid_search = GridSearchCV(perceptron, param_grid, cv=15, scoring='f1_macro', n_jobs=-1)

grid_search.fit(x_sa_train, y_sa_target_train)
best_params = grid_search.best_params_
print(best_params)

# best model search
best_perceptron = Perceptron(alpha=best_params["alpha"], max_iter=best_params["max_iter"])
best_perceptron.fit(x_sa_train, y_sa_target_train)
val_predicted = best_perceptron.predict(x_sa_validation)
evaluationMetrics(validation_target, val_predicted)
test_predicted = best_perceptron.predict(X_test_combined)


submission_data = {'preds': test_predicted, 'id': test_data['id']}
submission_df = pd.DataFrame(submission_data)
print(submission_df['preds'].value_counts())
submission_df.to_csv('perceptron_SA_submission.csv', index=False)

{'alpha': 0.1, 'max_iter': 2000}
Confusion Matrix:  [[3098  516]
 [ 270 1954]] 
roc_auc:  0.8679095185390149 
accuracy:  0.8653648509763617 
f1_score:  0.8325521942905837
0    2579
1    1921
Name: preds, dtype: int64


# **Sentiment Analysis + Naive Bayes**

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

# Define your parameter grid for GridSearchCV
param_grid = {
    'alpha': [0.1, 0.2, 0.3, 0.4]
}

# search best parameters
naive_bayes = MultinomialNB()
grid_search = GridSearchCV(estimator=naive_bayes, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(abs(x_sa_train),  y_sa_target_train)
best_params = grid_search.best_params_
print('best grid parameters', best_params)

## predict using the best classifier
best_NB = MultinomialNB(alpha=best_params['alpha'])
best_NB.fit(abs(x_sa_train), y_sa_target_train)
validation_pred = best_NB.predict(abs(x_sa_validation))
evaluationMetrics(validation_target, validation_pred)

# Save the results to a CSV file
test_predicted = best_NB.predict(abs(X_test_combined))
first_submission = {'preds': test_predicted, 'id': test_data['id']}
first_submission_df = pd.DataFrame(first_submission)
print(first_submission_df['preds'].value_counts())
first_submission_df.to_csv('nb_sa_submission.csv', index=False)

best grid parameters {'alpha': 0.1}
Confusion Matrix:  [[3473  141]
 [ 819 1405]] 
roc_auc:  0.7963648312119536 
accuracy:  0.8355601233299075 
f1_score:  0.7453580901856764
0    3549
1     951
Name: preds, dtype: int64
